In [34]:
# Import opencv for computer vision stuff
import cv2
# Import matplotlib so we can visualize an image
from matplotlib import pyplot as plt
import numpy as np
import socket


In [35]:
# Connect to capture device
cap = cv2.VideoCapture(0)

## Calculate the tiles in terms of x- and y-Steps

the picture will be divided in a set number of Tiles according to the steps. The result is a Dictionary with the boundaris for each tile

In [36]:
ret, frame = cap.read()

y = 0
yStep = 240

x = 0 
xStep = 160

height, width, _ = np.shape(frame)
print(height, width)

coord_x = [] 
coord_y = []

Tile_dict = {}

while x < width:
    coord_x.append([x, x+xStep])
    x += xStep
while y < height:
    coord_y.append([y, y+yStep])
    y += yStep

i = 0
while i < len(coord_x):
    tile_str = "tile" + str(i)
    Tile_dict[tile_str] = [coord_y[0][0], coord_y[0][1],coord_x[i][0],coord_x[i][1]]
    i += 1

j = 0

while j < len(coord_x):
    tile_str = "tile" + str(j+4)
    Tile_dict[tile_str] = [coord_y[1][0], coord_y[1][1],coord_x[j][0],coord_x[j][1]]
    j += 1


print(Tile_dict)

def create_rgb(color):
    red, green, blue = int(color[0]), int(color[1]), int(color[2])
    return (red, green, blue)

def create_bar(height, width, color):
    bar = np.zeros((height, width, 3), np.uint8)
    bar[:] = color
    red, green, blue = int(color[0]), int(color[1]), int(color[2])
    return bar, (red, green, blue)

for tileCoord in Tile_dict.values():
    tile = frame[tileCoord[0]:tileCoord[1],tileCoord[2]:tileCoord[3]]


480 640
{'tile0': [0, 240, 0, 160], 'tile1': [0, 240, 160, 320], 'tile2': [0, 240, 320, 480], 'tile3': [0, 240, 480, 640], 'tile4': [240, 480, 0, 160], 'tile5': [240, 480, 160, 320], 'tile6': [240, 480, 320, 480], 'tile7': [240, 480, 480, 640]}


## Function that returns the RGB values using a frame and the tiles boundaries

In [37]:
def tilesRGB(frame, Tile_dict = {}):
    k = 0
    tiles_rgb_values = []
    for tileCoord in Tile_dict.values():
        tile = frame[tileCoord[0]:tileCoord[1],tileCoord[2]:tileCoord[3]]

        img = tile
        height, width, _ = np.shape(img)
        # print(height, width)

        data = np.reshape(img, (height * width, 3))
        data = np.float32(data)

        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
        flags = cv2.KMEANS_RANDOM_CENTERS
        compactness, labels, centers = cv2.kmeans(data, 1, None, criteria, 10, flags)
        #print(centers)

        rgb_values = []

        for index, row in enumerate(centers):
            rgb = create_rgb(row)
            rgb_values.append(rgb)

        tiles_rgb_values.append(rgb_values[0])
        #print(rgb_values)
        k+=1

    return tiles_rgb_values


        

In [38]:
ret, frame = cap.read()
tilesRGB(frame, Tile_dict)

[(192, 189, 184),
 (207, 206, 196),
 (233, 232, 218),
 (222, 220, 210),
 (130, 130, 129),
 (117, 116, 125),
 (221, 216, 215),
 (219, 215, 215)]

In [39]:
print(frame)

[[[129 131 126]
  [128 129 126]
  [128 129 127]
  ...
  [228 229 209]
  [228 230 211]
  [230 232 213]]

 [[129 129 126]
  [128 129 126]
  [128 129 126]
  ...
  [226 229 210]
  [226 230 211]
  [226 230 211]]

 [[130 128 127]
  [128 129 126]
  [124 128 124]
  ...
  [224 228 210]
  [224 231 211]
  [224 231 211]]

 ...

 [[ 87  93  88]
  [ 88  94  89]
  [ 86  92  87]
  ...
  [189 187 186]
  [191 189 189]
  [190 188 188]]

 [[ 86  93  86]
  [ 85  91  86]
  [ 84  90  85]
  ...
  [191 189 189]
  [189 187 187]
  [188 186 186]]

 [[ 86  93  86]
  [ 84  91  85]
  [ 84  90  85]
  ...
  [190 188 188]
  [188 186 186]
  [188 186 186]]]


## Video and client that runs the video and sends the prominent RGB values for each tile to the Server (Pi)

In [41]:
# Connect to webcam
cap = cv2.VideoCapture(0)

    # create a socket object
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

server_ip = "192.168.88.101"  # replace with the server's IP address
server_port = 8000  # replace with the server's port number
    # establish connection with server
client.connect((server_ip, server_port))

# Loop through every frame until we close our webcam
while cap.isOpened(): 
    ret, frame = cap.read()
    
    # Show image 
    #print(tilesRGB(frame, Tile_dict), end="\r")

    font = cv2.FONT_HERSHEY_SIMPLEX
    bars = []
    rgb_values = []
    
    for index, row in enumerate(tilesRGB(frame, Tile_dict)):
        bar, rgb = create_bar(200, 200, row)
        bars.append(bar)
        rgb_values.append(rgb)

    img_bar = np.hstack(bars)

    for index, row in enumerate(rgb_values):
        image = cv2.putText(img_bar, f'{index + 1}. RGB: {row}', (5 + 200 * index, 200 - 10),
                            font, 0.5, (255, 0, 0), 1, cv2.LINE_AA)
        #print(f'{index + 1}. RGB{row}')

    cv2.imshow('Dominant colors', img_bar)
    cv2.imshow('Webcam', frame)
    
    # Checks whether q has been hit and stops the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        msg = "close"
        client.send(msg.encode("utf-8")[:1024])
        break
    else:
        result = tilesRGB(frame, Tile_dict)
        result_x = ""
        for x in result:
            str_x = str(x)
            result_x += str_x[1:-1] + "," 
        #print (result_x)

        client.send(result_x.encode("utf-8")[:1024])

            # receive message from the server
        response = client.recv(1024)
        response = response.decode("utf-8")

        print(f"Received: {response}", end="\r")


# Releases the webcam
client.close()
print("Connection to server closed")
cap.release()
# Closes the frame
cv2.destroyAllWindows()

Connection to server closed


## Raspberry Pi Server and Pixel Controller code

In [22]:
import socket

#include all neccessary packages to get LEDs to work with Raspberry Pi
import time
import board
import neopixel

#Initialise a strips variable, provide the GPIO Data Pin
#utilised and the amount of LED Nodes on strip and brightness (0 to 1 value)
pixels1 = neopixel.NeoPixel(board.D18, 55, brightness=1)

#Also create an arbitary count variable
x=0

#Focusing on a particular strip, use the command Fill to make it all a single colour
#based on decimal code R, G, B. Number can be anything from 255 - 0. Use a RGB Colour
#Code Chart Website to quickly identify a desired fill colour.
pixels1.fill((0, 220, 0))
#Sleep for three seconds, You should now have all LEDs showing light with the first node
#Showing a different colour
time.sleep(4)

#Complete the script by returning all the LED to off
pixels1.fill((0, 0, 0))



def run_server():
    # create a socket object
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    server_ip = "192.168.88.101"
    port = 8000

    # bind the socket to a specific address and port
    server.bind((server_ip, port))
    # listen for incoming connections
    server.listen(0)
    print(f"Listening on {server_ip}:{port}")

    # accept incoming connections
    client_socket, client_address = server.accept()
    print(f"Accepted connection from {client_address[0]}:{client_address[1]}")

    # receive data from the client
    while True:
        request = client_socket.recv(1024)
        request = request.decode("utf-8") # convert bytes to string

        list_request = request.split(",")
        #while x < 8:
        try: 
            int_test = int(list_request[0])
        except:
            print(str(list_request[0]))
        else:
            pixels1[0] = (int(list_request[2]),int(list_request[1]), int(list_request[0]))
            pixels1[1] = (int(list_request[5]),int(list_request[4]), int(list_request[3]))
            pixels1[2] = (int(list_request[8]),int(list_request[7]), int(list_request[6]))
            pixels1[3] = (int(list_request[11]),int(list_request[10]), int(list_request[9]))
            pixels1[4] = (int(list_request[14]),int(list_request[13]), int(list_request[12]))
            pixels1[5] = (int(list_request[17]),int(list_request[16]), int(list_request[15]))
            pixels1[6] = (int(list_request[20]),int(list_request[19]), int(list_request[18]))
            pixels1[7] = (int(list_request[23]),int(list_request[22]), int(list_request[21]))
        #print(list_request[:2])

        # if we receive "close" from the client, then we break
        # out of the loop and close the conneciton
        if request.lower() == "close":
            # send response to the client which acknowledges that the
            # connection should be closed and break out of the loop
            client_socket.send("closed".encode("utf-8"))
            break

        #print(f"Received: {request}", end="\r")

        response = "accepted".encode("utf-8") # convert string to bytes
        # convert and send accept response to the client
        client_socket.send(response)

    # close connection socket with the client
    pixels1.fill((0, 0, 0))
    client_socket.close()
    print("Connection to client closed")
    # close server socket
    server.close()


run_server()


ModuleNotFoundError: No module named 'board'